# El presente archivo es la primera parte sobre como se crea el sistema de recomendacion usando SVC para la clasificacion de los resultados de aprendizaje

## Importamos las librerias necesarias, para este caso usaremos SVC

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

## Iteramos la diferente data dividida entre los grados y asi tener un sistema de recomendacion para cada grado para que sea mas especifica la clasificacion

In [ ]:

grade_levels = df['level'].unique()

models = {}

for level in grade_levels:
    df_level = df[df['level'] == level]

    X, y = preprocess_data(df_level)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    svm = SVC(kernel='rbf')
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01, 0.001],
    }
    grid_search = GridSearchCV(svm, param_grid, cv=5)
    grid_search.fit(X_train, y_train)

    models[level] = grid_search.best_estimator_

    y_pred = models[level].predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy for level {level}: {accuracy:.2f}')

## Funcion que sirve para hacer la prediccion de los resultados por grado segun la nueva data y el nivel

In [ ]:
def predict_for_grade_level(level, new_student_data):
    model = models[level]

    normalized_score = new_student_data['score'] / avg_scores_by_level[level]
    normalized_time = new_student_data['time'] / avg_time_by_level[level]
    new_features = scaler.transform([[normalized_score, normalized_time]])

    prediction = model.predict(new_features)
    return prediction
